Exploring the dataset


In [ ]:
#Importing necessary libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import MarkerCluster
from folium.plugins import TimestampedGeoJson
import json
from datetime import datetime, timedelta

Loading the air quality dataset for Delhi

In [108]:
df = pd.read_csv(r'..\Datasets\r.k.-puram, delhi-air-quality.csv')

In [109]:
df.head()

,date,pm25,pm10,o3,no2,so2,co
0,2025/4/1,145,191,6,19,18,13
1,2025/4/2,182,130,5,19,15,12
2,2025/4/3,150,197,5,19,15,10
3,2025/4/4,153,151,6,17,13,9
4,2025/4/5,153,128,6,17,16,8


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4063 entries, 0 to 4062
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    4063 non-null   object
 1    pm25   4063 non-null   object
 2    pm10   4063 non-null   object
 3    o3     4063 non-null   object
 4    no2    4063 non-null   object
 5    so2    4063 non-null   object
 6    co     4063 non-null   object
dtypes: object(7)
memory usage: 222.3+ KB


Data Cleanup

In [111]:
#Renaming the columns for better readability
df = df.rename(columns=lambda x: x.strip())

In [112]:
columns = ['pm25', 'pm10', 'so2', 'co', 'o3', 'no2']

for i in columns:
    df[i] = pd.to_numeric(df[i], errors='coerce')
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4063 entries, 0 to 4062
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    4063 non-null   object 
 1   pm25    3840 non-null   float64
 2   pm10    3966 non-null   float64
 3   o3      3983 non-null   float64
 4   no2     3991 non-null   float64
 5   so2     3962 non-null   float64
 6   co      3934 non-null   float64
dtypes: float64(6), object(1)
memory usage: 222.3+ KB


In [113]:
df.isnull().sum()

date      0
pm25    223
pm10     97
o3       80
no2      72
so2     101
co      129
dtype: int64

In [114]:
df.describe()

,pm25,pm10,o3,no2,so2,co
count,3840.000000,3966.000000,3983.000000,3991.000000,3962.000000,3934.000000
mean,186.752344,153.195663,36.794376,23.919068,8.642605,12.771225
std,80.129749,99.513972,25.764097,13.730807,6.280451,9.214042
min,24.000000,10.000000,1.000000,1.000000,1.000000,1.000000
25%,135.000000,84.000000,17.000000,14.500000,4.000000,7.000000
50%,172.000000,129.000000,32.000000,22.000000,7.000000,11.000000
75%,225.000000,192.000000,50.000000,31.000000,11.000000,16.000000
max,824.000000,933.000000,195.000000,250.000000,96.000000,159.000000


Feature Engineering


In [115]:
#Replacing null values with mean of the column

for i in columns:
    df[i].fillna(df[i].mean(), inplace=True)

C:\Users\akw97\AppData\Local\Temp\ipykernel_10440\1202766690.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [ ]:
#Checking if the null values were replaced successfully
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4063 entries, 0 to 4062
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    4063 non-null   object 
 1   pm25    4063 non-null   float64
 2   pm10    4063 non-null   float64
 3   o3      4063 non-null   float64
 4   no2     4063 non-null   float64
 5   so2     4063 non-null   float64
 6   co      4063 non-null   float64
dtypes: float64(6), object(1)
memory usage: 222.3+ KB


In [117]:
#Changing date into datretime format
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')

In [ ]:
# Set date as index and resample for all pollutants
monthly_df = df.copy()
monthly_df = monthly_df.set_index('date')
monthly_df = monthly_df.resample('M').mean().reset_index()


C:\Users\akw97\AppData\Local\Temp\ipykernel_10440\3634961901.py:3: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [ ]:
#Checking the datafrme and its info
monthly_df.info()
monthly_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    137 non-null    datetime64[ns]
 1   pm25    137 non-null    float64       
 2   pm10    137 non-null    float64       
 3   o3      137 non-null    float64       
 4   no2     137 non-null    float64       
 5   so2     137 non-null    float64       
 6   co      137 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 7.6 KB


,date,pm25,pm10,o3,no2,so2,co
0,2013-12-31,186.752344,199.000000,20.000000,45.000000,4.000000,15.000000
1,2014-01-31,293.691745,286.373189,18.393146,37.897302,5.254753,23.059041
2,2014-02-28,231.250000,218.500000,30.000000,19.142857,3.951522,17.785714
3,2014-03-31,163.709677,124.419355,59.677419,23.000000,8.225806,9.516129
4,2014-04-30,181.925078,168.639855,58.993146,20.230636,10.521420,10.825708


Time Series Plot for PM 2.5 pollutant on a monthly basis

In [160]:
# Spike detection threshold
spike_threshold = 100
spikes = monthly_df[monthly_df['pm25'] > spike_threshold]

# Create figure
fig = go.Figure()

# Add solid line (constant color)
fig.add_trace(go.Scatter(
    x=monthly_df['date'],
    y=monthly_df['pm25'],
    mode='lines',
    line=dict(color='lightgray', width=2),
    name='Monthly PM2.5 Line'
))

# Add gradient-colored markers
fig.add_trace(go.Line(
    x=monthly_df['date'],
    y=monthly_df['pm25'],
    mode='markers',
    marker=dict(
        size=12,
        color=monthly_df['pm25'],
        colorscale='RdYlGn_r',
        colorbar=dict(title='PM2.5'),
        showscale=True
    ),
    name='PM2.5 Intensity'
))

# Spike annotations
for _, row in spikes.iterrows():
    if row['pm25'] > 200:
        fig.add_annotation(
            x=row['date'],
            y=row['pm25'],
            text=f"🚨 {row['pm25']:.1f}",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-30,
            bgcolor="red",
            font=dict(color="white")
        )

# Layout tweaks
fig.update_layout(
    title="🌫️ Monthly PM2.5 Levels",
    title_x=0.5,
    xaxis_title="Date",
    yaxis_title="PM2.5 (µg/m³)",
    template="plotly_white",
    hovermode='x unified',
    font=dict(family="Arial", size=14),
    height=600,
    margin=dict(l=50, r=50, t=60, b=40),
)

fig.show()


c:\Users\akw97\miniconda3\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [161]:
# Spike detection threshold
spike_threshold = 100
spikes = monthly_df[monthly_df['pm10'] > spike_threshold]

# Create figure
fig = go.Figure()

# Add solid line (constant color)
fig.add_trace(go.Scatter(
    x=monthly_df['date'],
    y=monthly_df['pm10'],
    mode='lines',
    line=dict(color='lightgray', width=2),
    name='Monthly PM10 Line'
))

# Add gradient-colored markers
fig.add_trace(go.Line(
    x=monthly_df['date'],
    y=monthly_df['pm10'],
    mode='markers',
    marker=dict(
        size=12,
        color=monthly_df['pm10'],
        colorscale='RdYlGn_r',
        colorbar=dict(title='PM10'),
        showscale=True
    ),
    name='PM10 Intensity'
))

# Spike annotations
for _, row in spikes.iterrows():
    if row['pm10'] > 200:
        fig.add_annotation(
            x=row['date'],
            y=row['pm10'],
            text=f"🚨 {row['pm10']:.1f}",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-30,
            bgcolor="red",
            font=dict(color="white")
        )

# Layout tweaks
fig.update_layout(
    title="🌫️ Monthly PM10 Levels",
    title_x=0.5,
    xaxis_title="Date",
    yaxis_title="PM10 (µg/m³)",
    template="plotly_white",
    hovermode='x unified',
    font=dict(family="Arial", size=14),
    height=600,
    margin=dict(l=50, r=50, t=60, b=40),
)

fig.show()


c:\Users\akw97\miniconda3\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




Importing the vehicles count dataset

Explorting the dataset

In [ ]:
df1 = pd.read_csv(r'..\Datasets\vehicles_data.csv')

df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   _id               12 non-null     int64  
 1   Type of Vehicles  12 non-null     object 
 2   2015-16           9 non-null      float64
 3   2016-17           9 non-null      float64
 4   2017-18           9 non-null      float64
 5   2018-19           9 non-null      float64
 6   2019-20           9 non-null      float64
 7   2020-21           9 non-null      float64
 8   2021-22           12 non-null     int64  
 9   2022-23           12 non-null     int64  
dtypes: float64(6), int64(3), object(1)
memory usage: 1.1+ KB


In [196]:
df1.head(10)

,_id,Type of Vehicles,2015-16,2016-17,2017-18,2018-19,2019-20,2020-21,2021-22,2022-23
0,1,Cars and Jeeps,2986579.0,3152710.0,3246637.0,3249670.0,3311579.0,3384736.0,2057657,2071115
1,2,Motor Cycles and Scooters,6104070.0,6607879.0,7078428.0,7556002.0,7959753.0,8239550.0,5135821,5294900
2,3,Auto Rickshaws,198137.0,105399.0,113074.0,113240.0,114891.0,114869.0,92149,93654
3,4,Taxis,91073.0,118308.0,118060.0,109780.0,122476.0,112401.0,85079,83278
4,5,Buses,34365.0,35206.0,35285.0,32218.0,33302.0,33294.0,17282,17232
5,6,Other Passenger Vehicles [E-Rickshaw(p)],6368.0,59759.0,76231.0,81422.0,85477.0,91887.0,104534,118506
6,7,Ambulances,2990.0,3059.0,3220.0,2358.0,2287.0,2289.0,1131,1172
7,8,"Tractors, All goods Vehicles & Others",281159.0,300437.0,315080.0,246861.0,263112.0,274324.0,245716,265739
8,9,Total Vehicles Plying,9704741.0,10382757.0,10986015.0,11391551.0,11892877.0,12253350.0,7739369,7945596
9,10,No. of Vehicles taken NOC,NaN,NaN,NaN,NaN,NaN,NaN,83240,623034


In [197]:
#Plotting the time series data for each year

df1["Type of Vehicles"].unique()

array(['Cars and Jeeps', 'Motor Cycles and Scooters', 'Auto Rickshaws',
       'Taxis', 'Buses', 'Other Passenger Vehicles [E-Rickshaw(p)]',
       'Ambulances', 'Tractors, All goods Vehicles & Others',
       'Total Vehicles Plying', 'No. of Vehicles taken NOC',
       'No. of Vehicles Deregistered', 'No. of Vehicles Scrapped'],
      dtype=object)

Data Cleanup

In [198]:
# Dropping the unnecessary rows
df1.drop(df1.index[2:], inplace=True)

In [199]:
#Checking to make sure changes were made
df1.head(10)

,_id,Type of Vehicles,2015-16,2016-17,2017-18,2018-19,2019-20,2020-21,2021-22,2022-23
0,1,Cars and Jeeps,2986579.0,3152710.0,3246637.0,3249670.0,3311579.0,3384736.0,2057657,2071115
1,2,Motor Cycles and Scooters,6104070.0,6607879.0,7078428.0,7556002.0,7959753.0,8239550.0,5135821,5294900


In [200]:
# Melt the dataframe excluding the '_id' column
df_long = df1.drop('_id', axis=1).melt(id_vars=["Type of Vehicles"], var_name="Year", value_name="Count")

# Convert 'Year' from '2015-16' to datetime (we'll use the starting year)
df_long['Year'] = df_long['Year'].str[:4].astype(int)
df_long['Year'] = pd.to_datetime(df_long['Year'], format='%Y')

In [201]:
df_long

,Type of Vehicles,Year,Count
0,Cars and Jeeps,2015-01-01,2986579.0
1,Motor Cycles and Scooters,2015-01-01,6104070.0
2,Cars and Jeeps,2016-01-01,3152710.0
3,Motor Cycles and Scooters,2016-01-01,6607879.0
4,Cars and Jeeps,2017-01-01,3246637.0
5,Motor Cycles and Scooters,2017-01-01,7078428.0
6,Cars and Jeeps,2018-01-01,3249670.0
7,Motor Cycles and Scooters,2018-01-01,7556002.0
8,Cars and Jeeps,2019-01-01,3311579.0
9,Motor Cycles and Scooters,2019-01-01,7959753.0


Plotting a line chart for each type of vehicle

In [ ]:
fig = px.line(
    df_long,
    x='Year',
    y='Count',
    color='Type of Vehicles',
    markers=True,
    title='📈 Vehicle Type Trends (2015–2022)',
    labels={'Year': 'Year', 'Count': 'Number of Vehicles'},
    template='plotly_white'
)


# Step 4: Annotate Peaks
for vehicle in df_long['Type of Vehicles'].unique():
    sub_df = df_long[df_long['Type of Vehicles'] == vehicle]
    peak_row = sub_df.loc[sub_df['Count'].idxmax()]
    
    fig.add_annotation(
        x=peak_row['Year'],
        y=peak_row['Count'],
        text=f"⬆️ Peak {vehicle}: {peak_row['Count']:,}",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40,
        bgcolor="lightyellow",
        font=dict(size=11),
        bordercolor="gray",
        borderwidth=1
    )


# Step 5: Layout Polishing
fig.update_layout(
    height=650,
    title_x=0.5,
    hovermode='x unified',
    font=dict(family="Arial", size=13),
    margin=dict(l=50, r=50, t=70, b=40),
    legend_title="Vehicle Type"
)

fig.show()

Mapping Geo Spatial Data from NASA Satelittes for Fires Near Delhi Area Using Keplar

In [110]:
#Importing the dataset
df_geo1 = pd.read_csv('../Datasets/fire_archive_M-C61_601386.csv')
df_geo2 = pd.read_csv('../Datasets/fire_archive_SV-C2_601390.csv')
df_geo3 = pd.read_csv('../Datasets/fire_archive_J1V-C2_601388.csv')

In [111]:
df_geo1.info()
df_geo2.info()
df_geo3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905405 entries, 0 to 905404
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   latitude    905405 non-null  float64
 1   longitude   905405 non-null  float64
 2   brightness  905405 non-null  float64
 3   scan        905405 non-null  float64
 4   track       905405 non-null  float64
 5   acq_date    905405 non-null  object 
 6   acq_time    905405 non-null  int64  
 7   satellite   905405 non-null  object 
 8   instrument  905405 non-null  object 
 9   confidence  905405 non-null  int64  
 10  version     905405 non-null  float64
 11  bright_t31  905405 non-null  float64
 12  frp         905405 non-null  float64
 13  daynight    905405 non-null  object 
 14  type        905405 non-null  int64  
dtypes: float64(8), int64(3), object(4)
memory usage: 103.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6528640 entries, 0 to 6528639
Data columns (total 15 columns)

In [112]:
df_geo1.head()
df_geo2.head()
df_geo3.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,22.58846,92.51312,337.59,0.63,0.72,2018-04-01,606,N20,VIIRS,n,2,299.01,6.89,D,0
1,22.50259,92.55136,330.26,0.63,0.72,2018-04-01,606,N20,VIIRS,n,2,299.59,8.80,D,0
2,25.07154,93.79645,326.75,0.47,0.64,2018-04-01,606,N20,VIIRS,n,2,281.35,8.20,D,0
3,25.10410,94.00828,347.82,0.45,0.63,2018-04-01,606,N20,VIIRS,n,2,293.19,7.43,D,0
4,25.10242,94.00741,343.47,0.45,0.63,2018-04-01,606,N20,VIIRS,n,2,293.91,5.23,D,0


In [113]:
#Checking for null values
df_geo1.isnull().sum()
df_geo2.isnull().sum()
df_geo3.isnull().sum()

latitude      0
longitude     0
brightness    0
scan          0
track         0
acq_date      0
acq_time      0
satellite     0
instrument    0
confidence    0
version       0
bright_t31    0
frp           0
daynight      0
type          0
dtype: int64

Feature Engineering


In [114]:
#Checking the acq_date column to acq_date time formatting
#Checking the acq_date column to acq_date time formatting
df_geo1['acq_date'] = pd.to_datetime(df_geo1['acq_date']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
df_geo2['acq_date'] = pd.to_datetime(df_geo2['acq_date']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
df_geo3['acq_date'] = pd.to_datetime(df_geo3['acq_date']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

In [115]:
#Keeping the rows starting from 2022 November 1st to 2023 April 30th

df_geo1 = df_geo1[(df_geo1['acq_date'] >= '2022-10-01') & (df_geo1['acq_date'] <= '2023-04-30')]
df_geo2 = df_geo2[(df_geo2['acq_date'] >= '2022-10-01') & (df_geo2['acq_date'] <= '2023-04-30')]
df_geo3 = df_geo3[(df_geo3['acq_date'] >= '2022-10-01') & (df_geo3['acq_date'] <= '2023-04-30')]


Now, the filtered datasets are ready to be used in Keplar for Geospatial Map

In [ ]:
#Creating a 'month' column for aggregation
df_geo1['month'] = pd.to_datetime(df_geo1['acq_date']).dt.strftime('%Y-%m')
df_geo2['month'] = pd.to_datetime(df_geo2['acq_date']).dt.strftime('%Y-%m')
df_geo3['month'] = pd.to_datetime(df_geo3['acq_date']).dt.strftime('%Y-%m')

In [136]:
#Concatenating the three dataframes into one

file_list = [df_geo1, df_geo2, df_geo3]
df_geo = pd.concat(file_list, ignore_index=True)
#Checking the concatenated dataframe
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094693 entries, 0 to 1094692
Data columns (total 16 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   latitude    1094693 non-null  float64
 1   longitude   1094693 non-null  float64
 2   brightness  1094693 non-null  float64
 3   scan        1094693 non-null  float64
 4   track       1094693 non-null  float64
 5   acq_date    1094693 non-null  object 
 6   acq_time    1094693 non-null  int64  
 7   satellite   1094693 non-null  object 
 8   instrument  1094693 non-null  object 
 9   confidence  1094693 non-null  object 
 10  version     1094693 non-null  float64
 11  bright_t31  1094693 non-null  float64
 12  frp         1094693 non-null  float64
 13  daynight    1094693 non-null  object 
 14  type        1094693 non-null  int64  
 15  month       1094693 non-null  object 
dtypes: float64(8), int64(2), object(6)
memory usage: 133.6+ MB


In [138]:
unique_months = df_geo['month'].unique()
unique_months

array(['2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
       '2023-04'], dtype=object)

In [141]:
# Loop through each month and plot
for month in unique_months:
    monthly_df = df_geo[df_geo['month'] == month]
    
    fig = px.scatter_mapbox(
        monthly_df,
        lat="latitude",
        lon="longitude",
        color="brightness",
        color_continuous_scale="YlOrRd",
        size_max=15,
        zoom=4.5,
        title=f"Brightness Map - {month}"
    )
    
    fig.update_layout(
        mapbox_style="carto-positron",
        margin={"r":0,"t":40,"l":0,"b":0}
    )
    
    # Save to HTML
    filename = f"brightness_map_{month}.html"
    fig.write_html(filename)
    print(f"Saved: {filename}")

Saved: brightness_map_2022-10.html
Saved: brightness_map_2022-11.html
Saved: brightness_map_2022-12.html
Saved: brightness_map_2023-01.html
Saved: brightness_map_2023-02.html
Saved: brightness_map_2023-03.html
Saved: brightness_map_2023-04.html
